In [70]:
from azureml.core import Workspace, Experiment

#ws = Workspace.get(name="udacity-project")
#exp = Experiment(workspace=ws, name="udacity-project")
#ws = Workspace.get(name="quick-starts-ws-148105",resource_group="quick-starts-ws-148105",subscription_id="81cefad3-d2c9-4f77-a466-99a7f541c7bb")
#print(ws)
ws = Workspace.from_config()
exp = Experiment(workspace=ws, name="udacity-project")


print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: quick-starts-ws-148105
Azure region: southcentralus
Subscription id: 81cefad3-d2c9-4f77-a466-99a7f541c7bb
Resource group: aml-quickstarts-148105


In [71]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

### YOUR CODE HERE ###
amlcomp_clus = "P1Cluster"
try:
    amlcomp = ComputeTarget(workspace=ws, name=amlcomp_clus)
except ComputeTargetException:
    comp_cfg = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2', max_nodes=4)
    
    amlcomp = ComputeTarget.create(ws, amlcomp_clus, comp_cfg)

amlcomp.wait_for_completion(show_output=True)


Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [72]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform, choice
import os
from azureml.core.environment import Environment
from azureml.core import ScriptRunConfig  
# Specify parameter sampler
ps = RandomParameterSampling({
        'C': uniform(0.05, 1),
        'max_iter': choice(50, 100)
        }
)

# Specify a Policy
bandpolicy = BanditPolicy(slack_factor = 0.1, evaluation_interval=1, delay_evaluation=5)

trainingscript_folder = "training"
if trainingscript_folder not in os.listdir():
    os.mkdir(f"./{trainingscript_folder}")

#This may be irrelevant
#https://docs.microsoft.com/en-us/azure/machine-learning/how-to-train-scikit-learn?view=azure-ml-py
#https://docs.microsoft.com/en-us/azure/machine-learning/how-to-migrate-from-estimators-to-scriptrunconfig?view=azure-ml-py
#TODO Figure out the source directory and arguments requirements
# Create a SKLearn estimator for use with train.py
#est = ### YOUR CODE HERE ###
myenv = Environment.get(workspace=ws, name="AzureML-Tutorial")
#myenv = Environment(workspace=ws, name="AzureML-Tutorial")
#myenv = Environment(workspace=ws, name="MyAzureML-Tutorial")

#myskenv = Environment.from_conda_specification(name='sklearn-env', file_path = './envs/conda_dependencies.yml')

# import shutil
# shutil.copy("./train.py", f"./{trainingscript_folder}")
# est = SKLearn(
#     source_directory = trainingscript_folder,
#     compute_target = amlcomp, 
#     entry_script="train.py"
# )

src = ScriptRunConfig(source_directory='.',
                      script='train.py',
                      #arguments=['--num_epochs', 30, '--output_dir', f"./{trainingscript_folder}"],
                      compute_target=amlcomp,
                      environment=myenv)

#https://docs.microsoft.com/en-us/azure/machine-learning/how-to-tune-hyperparameters
# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(run_config=src,
                             hyperparameter_sampling=ps,
                             policy=bandpolicy,
                             primary_metric_name="accuracy",
                             primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                             max_total_runs=20, ##Changed from 100
                             max_concurrent_runs=4)

In [73]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

### YOUR CODE HERE ###
hdrive_run = exp.submit(hyperdrive_config)
RunDetails(hdrive_run).show()

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

In [74]:
import joblib
from azureml.core.model import Model
# Get your best run and save the model from that run.

### YOUR CODE HERE ###
hdrive_run.wait_for_completion(show_output=True)
best_run = hdrive_run.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics()
parameter_values = best_run.get_details()['runDefinition']['arguments']

#print(hdrive_run.get_children_sorted_by_primary_metric())

print('Best Run Id: ', best_run.id)
print('\n Accuracy:', best_run_metrics['accuracy'])
print('\n --C - Inverse of regularization strength:',parameter_values[0])
print('\n --max_iter - Maximum number of iterations to converge:',parameter_values[1])
print('\n FileNames:', best_run.get_file_names())
print('----------------')
bestHdrivefolder="BestHdriveModel"
modelname = 'sklearn_hdrive_model'

os.makedirs(f'./{bestHdrivefolder}', exist_ok=True)
model = best_run.register_model(model_name='sklearn_hdrive',model_path='outputs/sklearn_hdrive_model.pkl')
print(model.name, model.id, model.version, sep='\t')
model.download(target_dir=f'./{bestHdrivefolder}/{modelname}.pkl', exist_ok = True)


RunId: HD_5641753d-8152-4430-815b-ac3da894d14d
Web View: https://ml.azure.com/runs/HD_5641753d-8152-4430-815b-ac3da894d14d?wsid=/subscriptions/81cefad3-d2c9-4f77-a466-99a7f541c7bb/resourcegroups/aml-quickstarts-148105/workspaces/quick-starts-ws-148105&tid=660b3398-b80e-49d2-bc5b-ac1dc93b5254

Execution Summary
RunId: HD_5641753d-8152-4430-815b-ac3da894d14d
Web View: https://ml.azure.com/runs/HD_5641753d-8152-4430-815b-ac3da894d14d?wsid=/subscriptions/81cefad3-d2c9-4f77-a466-99a7f541c7bb/resourcegroups/aml-quickstarts-148105/workspaces/quick-starts-ws-148105&tid=660b3398-b80e-49d2-bc5b-ac1dc93b5254

Best Run Id:  HD_5641753d-8152-4430-815b-ac3da894d14d_3

 Accuracy: 0.9121396054628225

 --C - Inverse of regularization strength: --C

 --max_iter - Maximum number of iterations to converge: 0.5028047159321426

 FileNames: ['azureml-logs/55_azureml-execution-tvmps_dd39187a9bbbc3c5822237025d886d46236a964cfff636b30550eb1593d56df2_d.txt', 'azureml-logs/65_job_prep-tvmps_dd39187a9bbbc3c58222370

'BestHdriveModel/sklearn_hdrive_model.pkl/sklearn_hdrive_model.pkl'

In [75]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

### YOUR CODE HERE ###
data = "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"
ds = TabularDatasetFactory.from_delimited_files(data, separator=',', header=True, encoding='utf8')

In [76]:
from train import clean_data

# Use the clean_data function to clean your data.
x, y = clean_data(ds)

In [79]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.


automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task='classification',
    primary_metric='accuracy',
    training_data=ds,
    label_column_name='y',
    n_cross_validations=3,
    compute_target=amlcomp,
    max_concurrent_iterations=4)

In [80]:
# Submit your automl run

### YOUR CODE HERE ###
autorun = exp.submit(config=automl_config, show_output=True)
RunDetails(autorun).show()

Submitting remote run.
No run_configuration provided, running on P1Cluster with default configuration
Running on remote compute: P1Cluster


Experiment,Id,Type,Status,Details Page,Docs Page
udacity-project,AutoML_ff5923d2-caa9-4737-b359-e8eb7643c7fd,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation



Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetFeaturization. Beginning to fit featurizers and featurize the dataset.
Current status: DatasetBalancing. Performing class balancing sweeping
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.
Current status: ModelSelection. Beginning model selection.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       ALERTED
DESCRIPTION:  To decrease model bias, please cancel the current run and fix balancing problem.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData
DETAILS:      Imbalanced data can lead to a falsely perceived positive effect of a model's accuracy because the input data has bias towards one class.
+---------------------------------+---------------------------------+-------------------------

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

In [87]:
# Retrieve and save your best automl model.


#autorun.wait_for_completion(show_output=True)

best_run_auto, fittedmodel = autorun.get_output()
print(fittedmodel.steps)
model_name = best_run_auto.properties['model_name']
description = 'AutoML for P1'
tags = None
bestAutofolder="BestAutoModel"
modelnameauto = 'auto_model'
#model = best_run_auto.register_model(model_name=model_name, description = description, tags = tags)
print(model.name, model.id, model.version, sep='\t')
model.download(target_dir=f'./{bestAutofolder}/{modelnameauto}.pkl', exist_ok = True)




[('datatransformer', DataTransformer(
    task='classification',
    is_onnx_compatible=False,
    enable_feature_sweeping=True,
    enable_dnn=False,
    force_text_dnn=False,
    feature_sweeping_timeout=86400,
    featurization_config=None,
    is_cross_validation=True,
    feature_sweeping_config={}
)), ('MaxAbsScaler', MaxAbsScaler(copy=True)), ('LightGBMClassifier', LightGBMClassifier(
    min_data_in_leaf=20,
    random_state=None,
    n_jobs=1,
    problem_info=ProblemInfo(
        dataset_samples=32950,
        dataset_features=132,
        dataset_classes=2,
        dataset_num_categorical=0,
        dataset_categoricals=None,
        pipeline_categoricals=None,
        dataset_y_std=None,
        dataset_uid=None,
        subsampling=False,
        task='classification',
        metric=None,
        num_threads=1,
        pipeline_profile='none',
        is_sparse=True,
        runtime_constraints={'mem_in_mb': None, 'wall_time_in_s': None, 'total_wall_time_in_s': 31449600, 

'BestAutoModel/auto_model.pkl/sklearn_hdrive_model.pkl'